In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import const
import datetime
import re
import sys
import jaconv


# 途中スタート位置
before_df = pd.read_csv('data_base_all.csv', index_col='URL', encoding='utf-8', low_memory=False)
before_t_id = '00000000' if len(before_df) == 0 else str(before_df.iloc[-1]['ID']).zfill(8)
skip = [int(before_t_id[0:2]), int(before_t_id[-6:])]

for i in range(1, 48):
    # 県ごと
    failed_cnt = 0
    for j in range(1000000):
        t_id = str(i*1000000+j).zfill(8)
        if failed_cnt >= 100:
            break
        if (i <= skip[0] - 1) or (i == skip[0] and j <= skip[1]):
            continue
        # df = df_blank.copy()
        df = pd.DataFrame()
    #     url = i[2]
        url = f'https://tabelog.com/{const.TODOFUKEN_URL_LIST[t_id[:2]]}/A0101/A010101/{t_id}/'
        shop_data = []
        response = requests.get(url)
        if response.status_code == 200:
            failed_cnt = 0
            print(f'load:{url} success!:{response.status_code}')
        else:
            print(f'load:{url} failed:{response.status_code}')
            failed_cnt += 1
            continue
        soup = BeautifulSoup(response.text, "html.parser")
        for key, selector in const.SELECTOR_DIC.items():
            elems_address = soup.find_all('p', class_='rstinfo-table__address')
            
            elems_price = soup.find_all('span', class_='rdheader-budget__price')
            elems_price_r = soup.find_all('em', class_='gly-b-dinner')
            elems_award = soup.find_all('div', class_='rstinfo-table-badge-award')
            if key in ['説明', '説明詳細']:
                elsms_attr = soup.find_all(selector[0], class_=selector[1])
            if key == 'ID':
                continue
            try:
                value = None
                if key in 'URL':
                    value = url
                elif key in ['取得日時']:
                    value = datetime.datetime.now()
                elif key in ['都道府県', '所在地', '施設名']:
                    value = elems_address[0].contents[selector].get_text()
                elif key in ['説明']:
                    value = elsms_attr[0].contents[0]
                elif key in ['説明詳細']:
                    value = elsms_attr[0].get_text()
                elif key in ['予算(夜)', '予算(昼)']:
                    value = elems_price[selector].get_text()
                elif key in ['ステータス']:
                    for selec in selector:
                        elems = soup.select(selec)
                        if len(elems) >= 1:
                            value = elems[0].contents[0]
                elif key in ['食べログアワード']:
                    value = ''
                    for elem in elems_award:
                        text = elem.contents[1].get_text()
                        if '受賞店' in text:
                            value += text.strip().replace('年', ' ').replace('受賞店', '') + '/'
                elif key in ['百名店']:
                    value = ''
                    for elem in elems_award:
                        text = elem.contents[1].get_text()
                        if '選出店' in text:
                            value += text.strip().replace('百名店', '').replace('選出店', '').replace(' ', '') + '/'
                elif key in ['席数']:
                    elems = soup.find_all(selector[0], class_=selector[1])
                    value = elems[1].find_all('p')[0].get_text()
                    value = value if re.search(r'席$', value) else ''
                elif key in ['ホームページ']:
                    elems = soup.find_all('a', rel='nofollow noopener')
                    value = elems[0].attrs['href']
                elif key in ['公式アカウント']:
                    elems = soup.find_all('a', rel='nofollow noopener')
                    value = elems[1].attrs['href']
                elif key in ['最寄り駅', '点数', 'オープン日', '予約可否']:
                    elems = soup.find_all(selector[0], class_=selector[1])
                    value = elems[0].get_text()
                elif key in ['口コミ数', '保存件数']:
                    elems = soup.find_all(selector[0], class_=selector[1])
                    value = elems[0].find_all('em', class_='num')[0].get_text()
                    value = '0' if value == ' - ' else value
                else:
                    elems = soup.select(selector)
                    value = elems[0].contents[0]
                if key not in ['取得日時']:
                    value = jaconv.z2h(value, kana=False, digit=True, ascii=True).replace('~', '～').replace(',', '').replace('\n', '').strip()
                
                df.loc[t_id, key] = value
            except Exception:
                # print(key, sys.exc_info())
                df.loc[t_id, key] = ''
                continue
        df.to_csv(f'data_base_all.csv', mode='a', header=None)
print('end')

In [ ]:
import pandas as pd
from IPython.display import display

todofuken = '岩手'
institution = ''
genre = ''

print('load csv')
before_df = pd.read_csv('data_base_all.csv', index_col='URL', encoding='utf-8', low_memory=False)
# before_t_id = '00000000' if len(before_df) == 1 else str(before_df.iloc[-1]['ID']).zfill(8)

# print('load completed')
# df = df[df.都道府県.str.contains(todofuken) & (df.ジャンル1.str.contains(genre) | df.ジャンル2.str.contains(genre) | df.ジャンル3.str.contains(genre))]
# df = df.sort_values(['点数', '口コミ数', '保存件数'], ascending=False)
# df.to_csv('result.csv', encoding='utf-8')

# print(f'HIT件数：{len(df)} 件')

In [ ]:
before_df[before_df.都道府県.str.contains('北海道|青森')]

In [ ]:
df = pd.read_csv('data_base_all.csv', encoding='utf-8')
df = df.sort_values(['点数', '口コミ数', '保存件数'], ascending=False)
df = df.iloc[0:10][['店名', '点数', '口コミ数', '保存件数', 'ジャンル1']]

for i in df.itertuples():
    print(i[1:6])
    
    
aaa = [1, 2, 3]
aaa.remove(2)
aaa

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

df = pd.DataFrame(
    [
        [11, None, 15, 4, 5],
        [2, 12, 10, 8, 16],
        [17, 24, 14, 13, 23],
        [19, 6, 18, None, 7],
        [3, 9, 20, 21, 1],
    ],
    index=["a", "b", "c", "d", "e"],
    columns=["A", "B", "C", "D", "E"],
)
# df = df.style.highlight_null("red")

df

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get('https://tabelog.com/hokkaido/A0101/A010101/12000509/')
soup = BeautifulSoup(response.text, "html.parser")

# elems = soup.select('#rst-data-head > table:nth-child(2) > tbody > tr:nth-child(6) > td > p > span:nth-child(2) > a:nth-child(1)')
# print(elems)
# elems = soup.select('#rst-data-head > table:nth-child(2) > tbody > tr:nth-child(6) > td > p > span:nth-child(2) > a:nth-child(2)')
# print(elems)
# elems = soup.select('#rst-data-head > table:nth-child(2) > tbody > tr:nth-child(6) > td > p > span:nth-child(3)')
# print(elems)


# elems = soup.select('#rst-data-head > table:nth-child(2) > tbody > tr:nth-child(3) > td > p > span:nth-child(2) > a:nth-child(1)')
# print(elems)
# elems = soup.select('#rst-data-head > table:nth-child(2) > tbody > tr:nth-child(3) > td > p > span:nth-child(2) > a:nth-child(2)')
# print(elems)
# elems = soup.select('#rst-data-head > table:nth-child(4) > tbody > tr:nth-child(1) > td > p')
# print(elems[0].contents[0])

elems = soup.find_all('span', class_='rdheader-rating__review-target')
value = elems[0].find_all('em', class_='num')[0].get_text()
value = 0 if value == ' - ' else value
value = jaconv.z2h(value, kana=False, digit=True, ascii=True).replace('~', '～').replace(',', '').replace('\n', '').strip()
print(value)

In [ ]:
df_input = pd.read_csv('input/input.csv', header=0)
for i in df_input.itertuples():
    print(i)

In [ ]:
import jaconv

jaconv.z2h('克美ビル　４Ｆ', kana=False, digit=True, ascii=True)

value = '4'
value = 0 if value == '-' else 0 if value == '' else value
value

In [ ]:
import pandas as pd


df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df = pd.DataFrame({'a': ['aaa', 'bbb', 'ccc'], 'b': [4, 5, 6]})
df[df.a.str.contains('aa|bb')]
df.a.replace('a', 'b')

In [ ]:
import re

aaa = '2021 Gold/2020 Silver/2019 Gold/2018 Bronze/2017 Gold/'
ret_str = ''
for i in aaa.split('/'):
    ret_str += i[2:6].replace(' ', '') + '/'
ret_str[0:-2]

bbb = 'ラーメン2020/ラーメン2019/ラーメン2018/'
ret_str = re.sub(r'\d', '', bbb.split('/')[0]) + ' '
for meiten in bbb.split('/'):
    ret_str += meiten[-2:] + '/'
ret_str[0:-2]

## import pandas as pd
import function as func


df_all = pd.read_csv('data_base_all.csv', encoding='utf-8').fillna('')
df_target = func.processing_data_frame(df_all)
df_target.iloc[0].URL

In [ ]:
import numpy as np

a = np.random.randint(0, 9, 5)
b = np.random.randint(0, 9, 5)
print(a)
print(b)
print(np.dot(a, b))

a = np.random.randint(0, 9, (3, 2))
b = np.random.randint(0, 9, (2, 3))
print(a)
print(b)
print(np.dot(a, b))

In [ ]:
import pandas as pd

def func(row):
    return row['a'] + row['b']

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df['c'] = df['a'].apply(lambda x: x * 2)
df['d'] = df.apply(func, axis=1)
df

In [ ]:
array_sample = np.random.normal(size=1000000)

def non_vectorize(array_sample):
    result = []
    for i in array_sample:
        result.append(i*i)
    return np.array(result)

%time non_vectorize_result = non_vectorize(array_sample)

def vectorize(array_sample):
    return array_sample*array_sample

%time vectorize_result = vectorize(array_sample)

print(non_vectorize(array_sample))
print(vectorize(array_sample))

aaa = [1, 2, 3, 4, 5]
non_vectorize(aaa)

%time _ = [i*i for i in array_sample]

In [ ]:
import pandas as pd


YOSAN_LIST = {
    '-': 0, '～￥999': 1, '￥1000～￥1999': 2, '￥2000～￥2999': 3, '￥3000～￥3999': 4, '￥4000～￥4999': 5, '￥5000～￥5999': 6, '￥6000～￥7999': 7, '￥8000～￥9999': 8, 
    '￥10000～￥14999': 9, '￥15000～￥19999': 10, '￥20000～￥29999': 11, '￥30000～￥39999': 12, '￥40000～￥49999': 13, '￥50000～￥59999': 14, '￥60000～￥79999': 15, '￥80000～￥99999': 15, '￥100000～': 16, 
}

df_all = pd.read_csv('data_base.csv', encoding='utf-8', low_memory=False).fillna('')
df_all['予算'] = df_all['予算(夜)'].apply(lambda x: YOSAN_LIST[x])
df_all[df_all.予算 == 5]

In [ ]:
import pandas as pd

df = pd.read_csv('data_base_all.csv', index_col='URL', encoding='utf-8', low_memory=False)

# df1 = pd.DataFrame({'ジャンル': [1, 2, 3]})
# df2 = pd.DataFrame({'ジャンル': [3, 4, 5]})
df2 = pd.concat([df.ジャンル1, df.ジャンル2, df.ジャンル3])
df3 = df2[~df2.duplicated(keep='last')]


In [ ]:
df3 = df2[~df2.duplicated(keep='last')]
df3.tolist()

In [ ]:
aaa = 'abcdef'
aaa[:2]